In [6]:
#CNN-A
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-CNN-Dataset-A-80-20.h5')
    with open('Scaler-Original-CNN-Dataset-A-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-CNN-Dataset-A-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder
    

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    prediction = model.predict(features_df)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label


def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
        
    }

    return features, None
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection" 
    initial_packets = sniff(iface=interface_name, count=150)
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                
                
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}") 
                print(f"F1-Score: {f1}")
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.743120009076779
Precision: 0.7451720704681213
Recall: 0.7437459378723289
F1-Score: 0.7444583211728508
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: kids-360
Accuracy: 0.747381585425477
Precision: 0.7563153178372957
Recall: 0.7449498088781196
F1-Score: 0.7505895413785774
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.7421074410828917
Precision: 0.7369817035166785
Recall: 0.7575688372297401
F1-Score: 0.7471334785558821
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7424159552737025
Precision: 0.7422344335776432
Recall: 0.7586652762063517
F1-Score: 0.7503599179735846
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.7320580407243356
Precision: 0.7233

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.73183151428507
Precision: 0.7230220095024428
Recall: 0.7193184719670027
F1-Score: 0.721165485896856
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7357093049908194
Precision: 0.7264051399392022
Recall: 0.7433994782977484
F1-Score: 0.7348040622043408
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7406129153816818
Precision: 0.7411788451075286
Recall: 0.7326212528976969
F1-Score: 0.7368752042544923
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Air-Droid
Accuracy: 0.7325112985722337
Precision: 0.7320492082379082
Recall: 0.7154954359687569
F1-Score: 0.7236776696249321
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Communication Monitori

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7404894355977356
Precision: 0.7337493939096
Recall: 0.7557802002982775
F1-Score: 0.7446018743825565
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: FamiSafe
Accuracy: 0.7358150552799153
Precision: 0.7412553816781293
Recall: 0.7304920693619791
F1-Score: 0.735834367921054
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7490348012306648
Precision: 0.7572792956794404
Recall: 0.7427108394997785
F1-Score: 0.7499243204858516
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: kids-360
Accuracy: 0.7362774767785507
Precision: 0.7409694877818653
Recall: 0.7557685445175361
F1-Score: 0.7482958530190995
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.746443339872

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.742305303484329
Precision: 0.7420132422168256
Recall: 0.7286281140769054
F1-Score: 0.7352597653843639
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7367228462084363
Precision: 0.7318080476411416
Recall: 0.7466321028860502
F1-Score: 0.73914575618689
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.7408938178794177
Precision: 0.7370992561620959
Recall: 0.7257160916558412
F1-Score: 0.731363383823289
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7422430486291571
Precision: 0.7496237024516181
Recall: 0.7434850092942543
F1-Score: 0.746541736713533
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7302298032

In [7]:
#CNN-B
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-CNN-Dataset-B-80-20.h5')
    with open('Scaler-Original-CNN-Dataset-B-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-CNN-Dataset-B-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder
    

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    prediction = model.predict(features_df)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label


def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
        
    }

    return features, None
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    csv_files = ['1-Update-3-All_together-Bi.csv', '2-Update-3-All_together-Bi.csv','3-Update-3-All_together-Bi.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  # Replace with the correct interface name
    initial_packets = sniff(iface=interface_name, count=150)
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                
                
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}") 
                print(f"F1-Score: {f1}")
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: mLite
Accuracy: 0.7013665868715504
Precision: 0.6978450864928467
Recall: 0.7109983135181823
F1-Score: 0.7043602995045154
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Eyezy
Accuracy: 0.707647491482414
Precision: 0.7079559584910113
Recall: 0.7131310250697165
F1-Score: 0.7105340689531887
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: kids-360
Accuracy: 0.7341121326020514
Precision: 0.7299903058207503
Recall: 0.7339254227193307
F1-Score: 0.7319525753231757
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: mLite
Accuracy: 0.7330931180084095
Precision: 0.7245284434068658
Recall: 0.7461227808476768
F1-Score: 0.7351670716787213
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7113806796070772
Precision: 0.718576515306314

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: mLite
Accuracy: 0.7075359543080468
Precision: 0.6988432567234293
Recall: 0.690039362907981
F1-Score: 0.6944134066130808
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Air-Droid
Accuracy: 0.7278009696880633
Precision: 0.7245000891304223
Recall: 0.7163647499713822
F1-Score: 0.7204094528778888
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Air-Droid
Accuracy: 0.7303200492447403
Precision: 0.7269150916097531
Recall: 0.7242398545409875
F1-Score: 0.7255750071452526
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Eyezy
Accuracy: 0.7180584272639016
Precision: 0.7242162112109568
Recall: 0.7157700270148163
F1-Score: 0.7199683487277324
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.7329645928543996
Precision: 0.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.7340934814951672
Precision: 0.7439319352823424
Recall: 0.7437328823583972
F1-Score: 0.7438323955035037
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: FamiSafe
Accuracy: 0.7061244961329617
Precision: 0.7159790124082609
Recall: 0.7126835130583516
F1-Score: 0.7143274618650914
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7323532723856616
Precision: 0.7286028389799085
Recall: 0.7510548479252402
F1-Score: 0.7396585024642017
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: kids-360
Accuracy: 0.7270326715728573
Precision: 0.7346705714999769
Recall: 0.7464654061232509
F1-Score: 0.7405210254922792
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7029741844654

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Eyezy
Accuracy: 0.7120748793797903
Precision: 0.7174686554862567
Recall: 0.7134288627266737
F1-Score: 0.7154430564180013
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Qustodio
Accuracy: 0.7009945115389526
Precision: 0.6934921347610111
Recall: 0.6829568578039918
F1-Score: 0.6881841780211054
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7346919502748787
Precision: 0.7424394655820625
Recall: 0.7284673012332616
F1-Score: 0.7353870225134863
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7087798525336226
Precision: 0.6993598702400666
Recall: 0.714111610633583
F1-Score: 0.7066587619312156


In [8]:
#LSTM-A
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-LSTM-Dataset-A-1-80-20.h5')
    with open('Scaler-Original-LSTM-Dataset-A-1-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-LSTM-Dataset-A-1-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder
    

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    # Reshape the input to match the expected shape for the LSTM model
    features_reshaped = features_df.values.reshape((features_df.shape[0], 1, features_df.shape[1]))
    
    prediction = model.predict(features_reshaped)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label



def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
        
    }

    return features, None   
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len
def main():
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  # Replace with the correct interface name
    initial_packets = sniff(iface=interface_name, count=150)
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)              
                
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}"  
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7875680706895838
Precision: 0.7910668318219471
Recall: 0.7898931638645359
F1-Score: 0.7904795621913344
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: TiSpy
Accuracy: 0.7785690861075762
Precision: 0.7763876319161352
Recall: 0.7914894542816585
F1-Score: 0.7838658125765978
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: TiSpy
Accuracy: 0.7550947491019625
Precision: 0.7536378241742371
Recall: 0.7659250123716588
F1-Score: 0.759731741158528
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Air-Droid
Accuracy: 0.7724832262885116
Precision: 0.7787096565115779
Recall: 0.7909135813688907
F1-Score: 0.7847641757773374
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7638393943952342
Precision: 0.7682839586711319


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7722465557093553
Precision: 0.7745911515869266
Recall: 0.7729692643715789
F1-Score: 0.773779358087348
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: mLite
Accuracy: 0.783283957651343
Precision: 0.7736997841931151
Recall: 0.7838097077391009
F1-Score: 0.7787219337891566
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7802432436535243
Precision: 0.771404220885306
Recall: 0.7947480314662614
F1-Score: 0.7829021541077356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Air-Droid
Accuracy: 0.7508123583257815
Precision: 0.7419404610443945
Recall: 0.7607412892295443
F1-Score: 0.7512232616967371
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Eyezy
Accuracy: 0.7667451863278023
Precision: 0.7576218

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Air-Droid
Accuracy: 0.7639137908204298
Precision: 0.7613550547709118
Recall: 0.7565067896328449
F1-Score: 0.7589231791800297
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7551658291984301
Precision: 0.7532003426866024
Recall: 0.7707675201496098
F1-Score: 0.7618826806858753
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Qustodio
Accuracy: 0.7760824694940629
Precision: 0.773559017450576
Recall: 0.7565328693649431
F1-Score: 0.7649512138947376
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Qustodio
Accuracy: 0.7534614414335888
Precision: 0.7593121588539999
Recall: 0.7394776704918054
F1-Score: 0.7492636731469438
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Communication Monitoring
Accurac

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Qustodio
Accuracy: 0.7697263452716105
Precision: 0.7603627190178269
Recall: 0.7616335966982882
F1-Score: 0.760997627262135
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Qustodio
Accuracy: 0.7836920011961022
Precision: 0.7903737960273706
Recall: 0.7651831020306963
F1-Score: 0.777574479934484
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: mLite
Accuracy: 0.7867425473384485
Precision: 0.7784412306151854
Recall: 0.8017156759505891
F1-Score: 0.7899070463158261
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: mLite
Accuracy: 0.7868907746404817
Precision: 0.7856450853369358
Recall: 0.796601699417487
F1-Score: 0.7910854566413746


In [9]:
#LSTM-B
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-LSTM-Dataset-B-80-20.h5')
    with open('Scaler-Original-LSTM-Dataset-B-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-LSTM-Dataset-B-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder
    

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    # Reshape the input to match the expected shape for the LSTM model
    features_reshaped = features_df.values.reshape((features_df.shape[0], 1, features_df.shape[1]))
    
    prediction = model.predict(features_reshaped)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label



def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
        
    }

    return features, None   
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len
def main():
    csv_files = ['1-Update-3-All_together-Bi.csv', '2-Update-3-All_together-Bi.csv','3-Update-3-All_together-Bi.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  # Replace with the correct interface name
    initial_packets = sniff(iface=interface_name, count=150)
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)              
                
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}"  
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.7788874703380793
Precision: 0.7711946108586465
Recall: 0.7868014378382784
F1-Score: 0.7789198556494547
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7687934544015387
Precision: 0.7728590868978721
Recall: 0.7597541479952996
F1-Score: 0.7662505891480791
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: FamiSafe
Accuracy: 0.7865912501788037
Precision: 0.7840136141135219
Recall: 0.7972799421380197
F1-Score: 0.7905911289205495
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Eyezy
Accuracy: 0.7687361810965399
Precision: 0.7623340327021985
Recall: 0.7823319661399153
F1-Score: 0.7722035483481133
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: FamiSafe
Accuracy: 0.7872040540623338
Precision: 0.7893434873673552
Recall

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Eyezy
Accuracy: 0.7662656479427951
Precision: 0.7744275592520292
Recall: 0.7475949528410467
F1-Score: 0.7607747323548427
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Eyezy
Accuracy: 0.7598511907560604
Precision: 0.7521293051864645
Recall: 0.7440800201739666
F1-Score: 0.7480830109674124
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.786928145290357
Precision: 0.7852839898168417
Recall: 0.7712145674768446
F1-Score: 0.7781856908445757
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.751362029734736
Precision: 0.7555009392069534
Recall: 0.7519164125732449
F1-Score: 0.753704414020892
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7602662866373054
Precision: 0.758246613455483
Recall: 0.757497

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: kids-360
Accuracy: 0.763877988789249
Precision: 0.7711846969834489
Recall: 0.764552092094492
F1-Score: 0.7678540719519471
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.7542750584927278
Precision: 0.7472810923369658
Recall: 0.7438090710365691
F1-Score: 0.7455410393651807
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: kids-360
Accuracy: 0.7615044176215745
Precision: 0.7582749091195032
Recall: 0.7739646905705679
F1-Score: 0.7660394699664606
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Air-Droid
Accuracy: 0.7628494295093631
Precision: 0.7618589997200378
Recall: 0.7649927758084274
F1-Score: 0.7634226718159888
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7528676132374655
Precision: 0.756300956

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: FamiSafe
Accuracy: 0.7575731204451016
Precision: 0.7604384456498405
Recall: 0.7613631810046418
F1-Score: 0.7609005323656948
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7655229833473376
Precision: 0.7703813936262899
Recall: 0.7736570825246043
F1-Score: 0.7720157633764921
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7638227085162451
Precision: 0.7715455961146058
Recall: 0.7687797363129504
F1-Score: 0.7701601829784896
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.7539549976386781
Precision: 0.7571313057537146
Recall: 0.7617938597407408
F1-Score: 0.7594554265652854


In [10]:
#FFNN-A
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-FFNN-Dataset-A-80-20.h5')
    with open('Scaler-Original-FFNN-Dataset-A-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-FFNN-Dataset-A-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder
    

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    prediction = model.predict(features_df)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label


def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
        
    }

    return features, None 
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  # Replace with the correct interface name
    initial_packets = sniff(iface=interface_name, count=150) 
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.6415811992873168
Precision: 0.6421537568705952
Recall: 0.6279586958698702
F1-Score: 0.6349769027810578
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.6403677746977555
Precision: 0.6487960802097485
Recall: 0.6595613159163396
F1-Score: 0.6541344095910403
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.6480995359376771
Precision: 0.6564659945390713
Recall: 0.6640178307836505
F1-Score: 0.6602203182163661
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.6761284717792568
Precision: 0.6741841809468471
Recall: 0.676880692162262
F1-Score: 0.6755297456501873
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.66285255

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.6553200910348741
Precision: 0.6563825012897367
Recall: 0.6554861622302938
F1-Score: 0.655934025546546
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.6568110070363015
Precision: 0.6635880731940726
Recall: 0.6408992165305853
F1-Score: 0.6520463320097044
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.6532122846495326
Precision: 0.6538958987828977
Recall: 0.6609599977584104
F1-Score: 0.6574089722389584
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.6225975175579345
Precision: 0.6207872693065543
Recall: 0.6054161832624365
F1-Score: 0.6130053840805648
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.65700667617

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.6491625552696165
Precision: 0.644283336286868
Recall: 0.6329068774070352
F1-Score: 0.6385444394462556
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.6565660500798787
Precision: 0.6596499674287704
Recall: 0.6750924559825017
F1-Score: 0.6672818796935004
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.6695878517108508
Precision: 0.6765426451372558
Recall: 0.6555803126080973
F1-Score: 0.6658965468810356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.6439608190577598
Precision: 0.6406038914203496
Recall: 0.6560117536719303
F1-Score: 0.6482162756717589
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.6723367380590443
Precision: 0.6723079438927102
Re

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.679239777658559
Precision: 0.6731874882334
Recall: 0.6792366952377297
F1-Score: 0.6761985631008995
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.6352022658275024
Precision: 0.639964323361911
Recall: 0.6179899299095102
F1-Score: 0.628785198365527
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.6396462110897917
Precision: 0.6305803304887894
Recall: 0.652227714380215
F1-Score: 0.6412213726486604
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.643051048483035
Precision: 0.6520580071701145
Recall: 0.6481096091707516
F1-Score: 0.6500778128485356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0

In [11]:
#FFNN-B
import random#FFNN-B
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-FFNN-Dataset-B-80-20.h5')
    with open('Scaler-Original-FFNN-Dataset-B-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-FFNN-Dataset-B-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder
    

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    prediction = model.predict(features_df)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label


def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
        
    }

    return features, None 
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    csv_files = ['1-Update-3-All_together-Bi.csv', '2-Update-3-All_together-Bi.csv','3-Update-3-All_together-Bi.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection" 
    initial_packets = sniff(iface=interface_name, count=150) 
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.5918010169520547
Precision: 0.5943404157110855
Recall: 0.6094437341138746
F1-Score: 0.6017973277658221
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.5973148843089255
Precision: 0.5942835718859204
Recall: 0.5862234113326167
F1-Score: 0.5902259753857017
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Eyezy
Accuracy: 0.5945303872252653
Precision: 0.5863452133062654
Recall: 0.6115000874516839
F1-Score: 0.5986585229107317
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.5919116615132295
Precision: 0.5918666254199303
Recall: 0.5950627059949327
F1-Score: 0.5934603626159283
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: kids-360
Accuracy: 0.590549210432513
Precision

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.6092686113061595
Precision: 0.6143367035603355
Recall: 0.6045614223359821
F1-Score: 0.6094098652002153
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: TiSpy
Accuracy: 0.6008110891455701
Precision: 0.5962408945817586
Recall: 0.5969020247746671
F1-Score: 0.5965712765094051
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.602768252977762
Precision: 0.5966124236788213
Recall: 0.5924688288760633
F1-Score: 0.59453340667918
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Air-Droid
Accuracy: 0.6170311784735408
Precision: 0.6124909004463417
Recall: 0.604865768850072
F1-Score: 0.6086544539592819
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.602092443413478
Precision: 0.61024063

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.6097850462335408
Precision: 0.609853399625107
Recall: 0.5931967205905544
F1-Score: 0.601409751131173
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.5984534787342037
Precision: 0.6076352073754855
Recall: 0.6166494960427497
F1-Score: 0.6121091660457053
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.6144866139372934
Precision: 0.6088110901287034
Recall: 0.6331839041107897
F1-Score: 0.6207583520087918
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.6053579475746466
Precision: 0.5968513528865828
Recall: 0.5872455620175153
F1-Score: 0.5920094947552212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Eyezy
Accuracy: 0.594240586

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: mLite
Accuracy: 0.6171472068841641
Precision: 0.6255681011846507
Recall: 0.6121243323982877
F1-Score: 0.6187732039353357
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: Air-Droid
Accuracy: 0.5911744829123248
Precision: 0.5844884145347236
Recall: 0.599397007573216
F1-Score: 0.5918488395769516
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.593279185686387
Precision: 0.594348876841395
Recall: 0.6066549811437955
F1-Score: 0.600438881650118
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.6112529753067034
Precision: 0.6153306565448832
Recall: 0.6266035417306922
F1-Score: 0.6209159378360903
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted Label: kids-360
Accuracy: 0.5

In [20]:
#CNN-LSTM-A
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-CNN-LSTM-Dataset-A-1-80-20.h5')
    with open('Scaler-Original-CNN-LSTM-Dataset-A-1-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-CNN-LSTM-Dataset-A-1-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    # Reshape the input to match the expected shape for the LSTM model
    features_reshaped = features_df.values.reshape((features_df.shape[0], features_df.shape[1], 1))
    
    prediction = model.predict(features_reshaped)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict

def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
       
     }

    return features, None 
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len    

def main():
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  
    initial_packets = sniff(iface=interface_name, count=150)
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.771328141419613
Precision: 0.7754110955331441
Recall: 0.7699391667391711
F1-Score: 0.7726654433632761
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7628138449511462
Precision: 0.7726350939627513
Recall: 0.7734503919460491
F1-Score: 0.7730425279886666
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7775935661374572
Precision: 0.7689771034393622
Recall: 0.7831648739907939
F1-Score: 0.7760061451517485
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.7841073661505945
Precision: 0.7815616355207321
Recall: 0.7900600518952378
F1-Score: 0.7857878664605173
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.7828534027111262

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7794426637765974
Precision: 0.775168095823718
Recall: 0.7897614780620087
F1-Score: 0.7823967433680257
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.7738228511835183
Precision: 0.7755789118664733
Recall: 0.7782973401037574
F1-Score: 0.7769357481084757
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7512488967405759
Precision: 0.7450699170705446
Recall: 0.7537041544654574
F1-Score: 0.7493621654233613
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7682693152144225
Precision: 0.7625946846750927
Recall: 0.7723038462198107
F1-Score: 0.7674185572879207
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7790846794989433
Precision: 0.7739161

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.7628246313873958
Precision: 0.7634194079951198
Recall: 0.768532923562222
F1-Score: 0.7659676315570386
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.778304103747516
Precision: 0.7713889913085882
Recall: 0.771380335392643
F1-Score: 0.7713846633263329
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7572148953391915
Precision: 0.7563769955947182
Recall: 0.7393459193720603
F1-Score: 0.7477644951535353
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.7885226375805594
Precision: 0.7833429484212628
Recall: 0.7727972806495222
F1-Score: 0.7780343815382389
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.7772497462141883
Precision: 0.7864623147420863
Recall: 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.773231488016556
Precision: 0.7740519836537846
Recall: 0.7770336074415619
F1-Score: 0.7755399297869239
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7588484711409933
Precision: 0.7525205746319613
Recall: 0.7519326719559846
F1-Score: 0.7522265084251469
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7787692681632242
Precision: 0.7731057742096838
Recall: 0.7935098115432029
F1-Score: 0.7831749189463951
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7755655373043456
Precision: 0.7709046753497938
Recall: 0.7790279953176967
F1-Score: 0.7749450478518598
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy

In [22]:
#CNN-LSTM-B
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    model = tf.keras.models.load_model('Original-CNN-LSTM-Dataset-B-1-80-20.h5')
    with open('Scaler-Original-CNN-LSTM-Dataset-B-1-80-20.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    with open('Label-Encoder-Original-CNN-LSTM-Dataset-B-1-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, scaler, label_encoder

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    
    # Reshape the input to match the expected shape for the LSTM model
    features_reshaped = features_df.values.reshape((features_df.shape[0], features_df.shape[1], 1))
    
    prediction = model.predict(features_reshaped)
    
    # Check if the prediction is a probability distribution
    if prediction.shape[1] > 1:
        predicted_class = prediction.argmax(axis=1)[0]  # Get the class with the highest probability
    else:
        predicted_class = prediction[0][0]  # Single output class case
    
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict

def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
       
     }

    return features, None 
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len    

def main():
    csv_files = ['1-Update-3-All_together-Bi.csv', '2-Update-3-All_together-Bi.csv','3-Update-3-All_together-Bi.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    model, scaler, label_encoder = load_pretrained_model()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  
    initial_packets = sniff(iface=interface_name, count=150)
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.784964180347035
Precision: 0.7817353735187051
Recall: 0.7924797767172215
F1-Score: 0.7870709085288776
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7857359829176463
Precision: 0.7842307132042883
Recall: 0.7883961499530016
F1-Score: 0.786307915056031
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Air-Droid
Accuracy: 0.7893813942543304
Precision: 0.7887255697430121
Recall: 0.7785305741188356
F1-Score: 0.783594912725843
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Eyezy
Accuracy: 0.7860010057416557
Precision: 0.794530422561461
Recall: 0.7926696452945751
F1-Score: 0.7935989431732647
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7742038795209222
Pr

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Eyezy
Accuracy: 0.7858938317065587
Precision: 0.7819149338241917
Recall: 0.7874185085474301
F1-Score: 0.7846570708037072
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7828787420942277
Precision: 0.7776099717409435
Recall: 0.7996838946256167
F1-Score: 0.788492473040529
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.7681496571151165
Precision: 0.7617197147542266
Recall: 0.7872917533118028
F1-Score: 0.7742946545253832
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Air-Droid
Accuracy: 0.767211738586846
Precision: 0.7616143282580052
Recall: 0.7522785962356708
F1-Score: 0.7569176768912218
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Parent Con

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7796797255843841
Precision: 0.7740460151611691
Recall: 0.7730267506718244
F1-Score: 0.7735360471533309
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: mLite
Accuracy: 0.7682229292797026
Precision: 0.7711416552225061
Recall: 0.7699003759175935
F1-Score: 0.7705205156569126
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.766864183492544
Precision: 0.7626785278558443
Recall: 0.7529793462473843
F1-Score: 0.7577979029622327
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7774017051980152
Precision: 0.7688130095120519
Recall: 0.7797481336791885
F1-Score: 0.7742419625484164
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: No

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7867704832069401
Precision: 0.7839681770166659
Recall: 0.7893062667261749
F1-Score: 0.786628165853934
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: Qustodio
Accuracy: 0.7687733967191872
Precision: 0.7675702021831088
Recall: 0.7661718168752099
F1-Score: 0.7668703720420705
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7820714683783976
Precision: 0.7890458202292255
Recall: 0.7959862060420837
F1-Score: 0.7925008181886652
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7847558968065885
Precision: 0.7825604597542308
Recall: 0.7853356981750901
F1-Score: 0.7839456228235909
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Label:

In [14]:
#FFNN-XGB-A
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""

def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_models():
    ffnn_model = keras.models.load_model('Original-FFNN-XGB-Dataset-A-80-20-FFNN-Model.h5')
    with open('Original-FFNN-XGB-Dataset-A-80-20-xgb-model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('Original-FFNN-XGB-Dataset-A-80-20-scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    with open('Original-FFNN-XGB-Dataset-A-80-20-label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    with open('Original-FFNN-XGB-Dataset-A-80-20-class_names.pkl', 'rb') as f:
        class_names = pickle.load(f)
    return ffnn_model, xgb_model, scaler, label_encoder, class_names

def predict_with_models(ffnn_model, xgb_model, scaler, label_encoder, class_names, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_scaled = scaler.transform(features_df)

    # Get intermediate output from FFNN
    intermediate_output = ffnn_model.predict(features_scaled)

    # Predict with XGBoost
    xgb_predictions = xgb_model.predict(intermediate_output)
    
    # Map predicted class indices to class names
    predicted_class_name = class_names[xgb_predictions[0]]
    
    return predicted_class_name

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tp, tn, fp, fn = 0, 0, 0, 0
    if cm.size == 4:
        tn, fp, fn, tp = cm.ravel()
    
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict

def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
       'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0, 
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
    }

    return features, None

def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False
def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    # Corrected the function name to load_pretrained_models
    ffnn_model, xgb_model, scaler, label_encoder, class_names = load_pretrained_models()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  
    initial_packets = sniff(iface=interface_name, count=150) 
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_models(ffnn_model, xgb_model, scaler, label_encoder, class_names, features)
                
                
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
               
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
            
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.773428686984403
Precision: 0.7658489786310106
Recall: 0.7606961377662894
F1-Score: 0.76326386151203
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.7875433508761842
Precision: 0.7797011468148568
Recall: 0.7792684371503996
F1-Score: 0.7794847319308941
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.767345046708465
Precision: 0.7721571246917833
Recall: 0.7692491645783955
F1-Score: 0.7707004016100739
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.775164031441563
Precision: 0.7832942080078499
Recall: 0.7790009861969407
F1-Score: 0.7811416981680626
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7725215705876781
P

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7639426960308914
Precision: 0.7648161969425786
Recall: 0.7484843890729135
F1-Score: 0.7565621651266242
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.7657004241020575
Precision: 0.7573410816288777
Recall: 0.7490725930025466
F1-Score: 0.7531841450415663
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.7641869307971452
Precision: 0.7700391681355742
Recall: 0.7695396531194293
F1-Score: 0.7697893295938943
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7668350958935186
Precision: 0.7651456555932397
Recall: 0.7667007511267084
F1-Score: 0.7659224140112156
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.7814309655914025
Precision: 0.7847370791738191
R

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.784216916335071
Precision: 0.7910488482412488
Recall: 0.7888597756461375
F1-Score: 0.789952795388003
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.7719019370331376
Precision: 0.7651397954659039
Recall: 0.7878479934269229
F1-Score: 0.7763278718098141
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7984482560417706
Precision: 0.7977009179061617
Recall: 0.7967423228995519
F1-Score: 0.7972213322444693
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.7963981024332445
Precision: 0.7910738852153484
Recall: 0.8140676264215496
F1-Score: 0.8024060625092222
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7663172349380264


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7979100367043864
Precision: 0.8065375677008776
Recall: 0.8124753940150674
F1-Score: 0.809495592192418
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7872481624456985
Precision: 0.7826109513027907
Recall: 0.7990463569925248
F1-Score: 0.7907432618952497
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7903184442067421
Precision: 0.7811055946690638
Recall: 0.8042405529562534
F1-Score: 0.7925042695752523
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.7855571272578936
Precision: 0.7799343668631648
Recall: 0.7983950330572452
F1-Score: 0.7890567389108999
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.7

In [15]:
#FFNN-XGB-B
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""

def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_models():
    ffnn_model = keras.models.load_model('Original-FFNN-XGB-Dataset-B-1-80-20-FFNN-Model.h5')
    with open('Original-FFNN-XGB-Dataset-B-1-80-20-xgb-model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('Original-FFNN-XGB-Dataset-B-1-80-20-scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    with open('Original-FFNN-XGB-Dataset-B-1-80-20-label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    with open('Original-FFNN-XGB-Dataset-B-1-80-20-class_names.pkl', 'rb') as f:
        class_names = pickle.load(f)
    return ffnn_model, xgb_model, scaler, label_encoder, class_names

def predict_with_models(ffnn_model, xgb_model, scaler, label_encoder, class_names, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_scaled = scaler.transform(features_df)

    # Get intermediate output from FFNN
    intermediate_output = ffnn_model.predict(features_scaled)

    # Predict with XGBoost
    xgb_predictions = xgb_model.predict(intermediate_output)
    
    # Map predicted class indices to class names
    predicted_class_name = class_names[xgb_predictions[0]]
    
    return predicted_class_name

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tp, tn, fp, fn = 0, 0, 0, 0
    if cm.size == 4:
        tn, fp, fn, tp = cm.ravel()
    
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict

def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
       'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0, 
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
    }

    return features, None

def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False
def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    csv_files = ['1-Update-3-All_together-Bi.csv', '2-Update-3-All_together-Bi.csv','3-Update-3-All_together-Bi.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    # Corrected the function name to load_pretrained_models
    ffnn_model, xgb_model, scaler, label_encoder, class_names = load_pretrained_models()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  
    initial_packets = sniff(iface=interface_name, count=150) 
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_models(ffnn_model, xgb_model, scaler, label_encoder, class_names, features)
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")     
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
          0)
if __name__ == '__main__':
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7762060961374001
Precision: 0.776081859745392
Recall: 0.7922490182143886
F1-Score: 0.7840821093022582
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.7685471257076635
Precision: 0.7598157775648778
Recall: 0.7630363947945014
F1-Score: 0.7614226806044382
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.7627580860988271
Precision: 0.7607339989730519
Recall: 0.7637240134745168
F1-Score: 0.7622260739728971
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7704146100462959
Precision: 0.7781416450996043
Recall: 0.76245108114949
F1-Score: 0.7702164608269191
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7618548330270711
P

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.7690992908694659
Precision: 0.7617917610740323
Recall: 0.7758600783027672
F1-Score: 0.768761562613352
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.7641377550119584
Precision: 0.7650160976922818
Recall: 0.7476540971221712
F1-Score: 0.7562354593417614
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7643849600536518
Precision: 0.7684305612907719
Recall: 0.758625147530487
F1-Score: 0.7634963735227827
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7621620330047858
Precision: 0.7717448927231101
Recall: 0.7556807028258755
F1-Score: 0.7636283228914587
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7620743694024961
Precision: 0.761327238166

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.7693945563864685
Precision: 0.767028166638684
Recall: 0.7826238011544673
F1-Score: 0.7747475069801438
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.7647063013914766
Precision: 0.7547177385114825
Recall: 0.7633201684113723
F1-Score: 0.7589945793005963
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7742441082917306
Precision: 0.7694485914507402
Recall: 0.7701335135614752
F1-Score: 0.7697909001536296
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7634449345578594
Precision: 0.7604854908221262
Recall: 0.7450369790278323
F1-Score: 0.7526819745614053
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7769608845113

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7736295784530972
Precision: 0.7642155712591455
Recall: 0.7879385128576386
F1-Score: 0.7758957527251216
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.7634562164577975
Precision: 0.7674229223222141
Recall: 0.7827116682363924
F1-Score: 0.7749918999707114
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.7657982560457024
Precision: 0.7673702741345763
Recall: 0.7479304341548089
F1-Score: 0.7575256569884212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.7651305107510321
Precision: 0.7652211774204253
Recall: 0.7500978725978155
F1-Score: 0.757584057618687
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Parent Control
Accuracy:

In [17]:
#LSTM-XGB-A
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_models():
    lstm_model = keras.models.load_model('Original-LSTM-XGB-Dataset-A-1-80-20-LSTM-Model.h5')
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-XGB-model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-class_names.pkl', 'rb') as f:
        class_names = pickle.load(f)
    return lstm_model, xgb_model, scaler, label_encoder, class_names
def predict_with_models(lstm_model, xgb_model, scaler, label_encoder, class_names, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_scaled = scaler.transform(features_df)
    features_scaled_reshaped = features_scaled.reshape((features_scaled.shape[0], 1, features_scaled.shape[1]))
    intermediate_output = lstm_model.predict(features_scaled_reshaped)
    xgb_predictions = xgb_model.predict(intermediate_output)
    
    # Map predicted class indices to class names
    predicted_class_name = class_names[xgb_predictions[0]]
    
    return predicted_class_name

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
       
    }

    return features, None  
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    lstm_model, xgb_model, scaler, label_encoder, class_names = load_pretrained_models()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  
    initial_packets = sniff(iface=interface_name, count=150) 
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1
        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_models(lstm_model, xgb_model, scaler, label_encoder, class_names, features)
               
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
               
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
                
            
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.786784610833189
Precision: 0.7828291100441318
Recall: 0.7699146524838987
F1-Score: 0.7763181752958768
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.7966861679207604
Precision: 0.7939348550388127
Recall: 0.7898821989019214
F1-Score: 0.7919033420212206
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7858969215971522
Precision: 0.7876719505142771
Recall: 0.7884549574250924
F1-Score: 0.7880632594739817
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.774301866636338
Precision: 0.7659787240967231
Recall: 0.7807263555302735
F1-Score: 0.7732822314412803
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: TiSpy
Accuracy: 0.7931291290498422
Precision: 0.7935290876846371
Reca

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7613871182381178
Precision: 0.7611444038084062
Recall: 0.7555584547285223
F1-Score: 0.7583411428676219
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7965803468544059
Precision: 0.798912918025744
Recall: 0.7835513041569535
F1-Score: 0.7911575504228212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.775872665176276
Precision: 0.7827185433675865
Recall: 0.7771515526914256
F1-Score: 0.7799251140660092
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7837027065698792
Precision: 0.7762726384804701
Recall: 0.7955955321807995
F1-Score: 0.7858153176668056
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: All Tracker Communication Monitoring
Accuracy: 0.76998383

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.789623965063516
Precision: 0.784243018719278
Recall: 0.7721463876953326
F1-Score: 0.7781476942513441
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7751873052758811
Precision: 0.7788381432654499
Recall: 0.7861946295736181
F1-Score: 0.7824990967206403
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7921541214115747
Precision: 0.7993070608023733
Recall: 0.8087251756829246
F1-Score: 0.8039885376737118
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7952370750397062
Precision: 0.8040093368796118
Recall: 0.7844105018923909
F1-Score: 0.7940890085526434
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Qustodio
Accuracy: 0.7623130382296568
Precision: 0.762342141133

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.7649280844122589
Precision: 0.7646203450946102
Recall: 0.7574232859830011
F1-Score: 0.7610047997125841
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Air-Droid
Accuracy: 0.762097131438227
Precision: 0.7571613654973988
Recall: 0.7643341707023079
F1-Score: 0.7607308606777533
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.7909674434201011
Precision: 0.785353411665326
Recall: 0.774203139067853
F1-Score: 0.7797384151323006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: FamiSafe
Accuracy: 0.7663060302883287
Precision: 0.7574481255044592
Recall: 0.7626222393490881
F1-Score: 0.7600263764350589
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: kids-360
Accuracy: 0.7807671847465432
Precision: 0.7903380149224355
Reca

In [18]:
#LSTM-XGB-B
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_models():
    lstm_model = keras.models.load_model('Original-LSTM-XGB-Dataset-B-1-80-20-LSTM-Model.h5')
    with open('Original-LSTM-XGB-Dataset-B-1-80-20-XGB-model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-B-1-80-20-scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-B-1-80-20-label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-B-1-80-20-class_names.pkl', 'rb') as f:
        class_names = pickle.load(f)
    return lstm_model, xgb_model, scaler, label_encoder, class_names
def predict_with_models(lstm_model, xgb_model, scaler, label_encoder, class_names, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_scaled = scaler.transform(features_df)
    features_scaled_reshaped = features_scaled.reshape((features_scaled.shape[0], 1, features_scaled.shape[1]))
    intermediate_output = lstm_model.predict(features_scaled_reshaped)
    xgb_predictions = xgb_model.predict(intermediate_output)
    
    # Map predicted class indices to class names
    predicted_class_name = class_names[xgb_predictions[0]]
    
    return predicted_class_name

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
       
    }

    return features, None  
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    csv_files = ['1-Update-3-All_together-Bi.csv', '2-Update-3-All_together-Bi.csv','3-Update-3-All_together-Bi.csv']
    # Load the saved model and encoders
    with open('knn_model_with_encoders-A-11.pkl', 'rb') as file:
        data = pickle.load(file)
    
    knn = data['model']  # Loaded kNN model
    label_encoder = data['label_encoder']  # Loaded label encoder
    protocol_encoder = data['protocol_encoder']  # Loaded protocol encoder
    lstm_model, xgb_model, scaler, label_encoder, class_names = load_pretrained_models()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)
    interface_name = "Bluetooth Network Connection"  
    initial_packets = sniff(iface=interface_name, count=150) 
    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1
        if packet_count % 150 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                X_features = np.array([features[feature] for feature in features.keys()]).reshape(1, -1)
                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    # Use the saved kNN model for prediction
                    nearest_neighbor_label = knn.predict(X_features)[0]
                    label = label_encoder.inverse_transform([nearest_neighbor_label])[0]

                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        true_labels.append(label)
                    else:
                        label = 'Normal-Traffic'
                        true_labels.append(label)
                    predicted_label = predict_with_models(lstm_model, xgb_model, scaler, label_encoder, class_names, features)
               
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
            captured_packets = []
            packet_count = 0
    scapy.sniff(iface=interface_name, prn=process_packet, store=False, timeout=180)
if __name__ == '__main__':
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: TiSpy
Accuracy: 0.7611886166100114
Precision: 0.7659836583232709
Recall: 0.7585476161205861
F1-Score: 0.762247502230352
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: TiSpy
Accuracy: 0.7718296715533906
Precision: 0.7790807128796773
Recall: 0.7562743573032304
F1-Score: 0.7675081508673157
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: mLite
Accuracy: 0.7599879923510788
Precision: 0.7653136495433819
Recall: 0.7492067507531636
F1-Score: 0.7571745518504395
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7760028647087723
Precision: 0.7833061137302466
Recall: 0.7754453556750833
F1-Score: 0.7793559138657917
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7873051544477305


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: TiSpy
Accuracy: 0.7675855638007186
Precision: 0.7736164052645321
Recall: 0.7485296537420632
F1-Score: 0.7608662999656733
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Air-Droid
Accuracy: 0.7730066915348868
Precision: 0.7746286536231032
Recall: 0.7741438975806999
F1-Score: 0.7743861997391094
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Air-Droid
Accuracy: 0.7521667136581311
Precision: 0.7518645471699048
Recall: 0.7405361207482183
F1-Score: 0.7461573383855704
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: mLite
Accuracy: 0.7743968781148299
Precision: 0.7676555989234319
Recall: 0.7931571211970783
F1-Score: 0.7801980302491606
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Eyezy
Accuracy: 0.78883377449186
Precision: 0.7789443338831478
Recal

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: FamiSafe
Accuracy: 0.7890479943585383
Precision: 0.7911629386394645
Recall: 0.7697275993518912
F1-Score: 0.7802980857822526
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: FamiSafe
Accuracy: 0.7768978917691336
Precision: 0.780004538145173
Recall: 0.779029714384623
F1-Score: 0.7795168214988665
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Normal-Traffic
Accuracy: 0.7697733516045145
Precision: 0.7775867993206672
Recall: 0.7591974952093765
F1-Score: 0.7682821232014947
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: FamiSafe
Accuracy: 0.7573960165809014
Precision: 0.7532046971000688
Recall: 0.7720824204318616
F1-Score: 0.7625267386492488
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Air-Droid
Accuracy: 0.7671534633844751
Precision: 0.76566965

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: All Tracker Business Device Monitoring
Accuracy: 0.7638071685344562
Precision: 0.7633006608949652
Recall: 0.7673346571368186
F1-Score: 0.7653123432083029
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: kids-360
Accuracy: 0.773531190600569
Precision: 0.7656288698864193
Recall: 0.7582683167268828
F1-Score: 0.7619308172574234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: All Tracker Parent Control
Accuracy: 0.7541343807359592
Precision: 0.7485057219122391
Recall: 0.7543587488959881
F1-Score: 0.7514208378611088
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Eyezy
Accuracy: 0.7565004695970282
Precision: 0.7609603487800877
Recall: 0.7570038269672873
F1-Score: 0.7589769316041571
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: TiSpy
Accuracy: 0.77